In [1]:
import os
import requests
import datetime

import numpy as np
import pandas as pd

import customsettings

from typing import Callable

from bs4 import BeautifulSoup

from propscraper import PropScraper
from params import mode, site

In [2]:
# Returns current date as string in desired format for files
def date_path() -> str:
    return '.'.join([
        datetime.datetime.now().strftime("%m%d%y"),
        # (datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%m%d%y"),
        'csv'
    ])

In [3]:
# In case webpage goes down again
def save_directory():
    
    df_data: dict[str, list[str,...]] = {
        'team': list(),
        'name': list(),
        'url': list()
    }

    for team, player_links in directory.items():
        for name,url in player_links.items():
            df_data['team'].append(team)
            df_data['name'].append(name)
            df_data['url'].append(url)
            
    df: pd.DataFrame = pd.DataFrame(df_data)
    df.to_csv('../data/url-directory.csv', index=False)
    
    return None


def load_directory():
    df: pd.DataFrame = pd.read_csv('../data/url-directory.csv')
    
    team_dfs: dict[str, pd.DataFrame] = {
        team: (df
               .loc[df['team']==team]
               .set_index('name')
               .drop(['team'], axis=1)
               .T
               .to_dict()
              )
        for team in df['team'].drop_duplicates()
    }
    
    directory={team: dict() for team in team_dfs}
    
#     Improve this
    for team in team_dfs:
        for name in team_dfs[team]:
            directory[team][name] = team_dfs[team][name]['url']
        
    
    return directory

In [4]:
Props = PropScraper()
directory: dict[str,dict[str,str]] = Props.create_webpage_directory()

In [5]:
def scrape_props(name: str, team: str, site: str, **kwargs):
    try:
        return Props.scrape_player_props(
            name,
            directory[team][name],
            site,
            **kwargs
        )
    
    except KeyError:
        return (0.0, 0.0)

In [6]:
def check_site():
    try:
        assert(len(directory))
    except AssertionError:
        return 'ScoresAndOdds.com is down, or at least the page containing links is empty...'
    
    return 'No Issues'
    

In [7]:
check_site()

'No Issues'

In [8]:
if len(directory):
    save_directory()
else:
    directory = load_directory()
# directory = load_directory()

In [9]:
def scrape_fanduel(**kwargs):
    
    path: str = '../data/current-fanduel.csv'
    if mode == 'single-game':
        path: str = path.replace('.csv', '-sg.csv')
    
    
    columns: dict[str, str] = {
        'Nickname': 'name',
        'Position': 'pos',
        'Team': 'team',
        'Salary': 'salary',
        'Injury Indicator': 'injury',
    }
    
    MIN_SAL: int = 3_500 if kwargs.get('drop_minimums', False) else 0
    
    keep_minimums: tuple[str,...] = tuple()
    drop_minimums: tuple[str,...] = tuple([
        name for name in (pd.read_csv(path, usecols=['Nickname','Salary']).pipe(lambda df_: df_.loc[df_['Salary'] == MIN_SAL]['Nickname'])) if name not in keep_minimums
    ])
    
    df: pd.DataFrame = (pd
                        .read_csv(path, usecols=columns)
                        .rename(columns,axis=1)
                        .pipe(lambda df_: df_.loc[df_['injury']!='O'])
                        .drop('injury', axis=1)
                        .assign(name=lambda df_: df_.name.str.replace('.','',regex=False))
                        # .pipe(lambda df_: df_.loc[(df_['name'].isin(drop_minimums) == False)])
                       )

    # scoresandodds : FanDuel
    name_issues = {
        'Moe Wagner': 'Moritz Wagner',
        'Moritz Wagner': 'Moe Wagner'
    }


    
    df['name'] = df['name'].map(lambda name: name_issues.get(name, name))
    df['input'] = df.loc[:,['name','team']].apply(tuple, axis=1)
    df['output'] = df['input'].apply(lambda x: scrape_props(*x, 'fanduel'))
    
    df['fpts'] = df['output'].map(lambda x: x[0])
    df['e_fpts'] = df['output'].map(lambda x: x[1])
    
    
    for col in ('fpts', 'e_fpts'):
        df[f'{col}/$'] = 1000 * (df[col] / df['salary'])
    
    df['5x'] = 5 * (df['salary'] / 1000)
    df['value'] = df['fpts'] - df['5x']
    
    df = (df
          .loc[df['fpts']>0.0]
          .drop(['input', 'output', '5x'], axis=1)
          .assign(fpts_1k=lambda df_: 1000 * df_.fpts / df_.salary)
          .rename({'fpts_1k': 'fpts-1k'}, axis=1)
          .sort_values('value', ascending=False)
          .set_index('name')
          .round(2)
         )
    
    single_game: bool = 'sg' in path or len(df['team'].drop_duplicates()) == 2
    
    df.to_csv(f'../data/fanduel-props{"-sg" if single_game else ""}.csv')
    
    # Save to optimizer
    df.to_csv('/home/deegs/devel/repos/nba-boxscores-git/nba-boxscores/data/2023-2024/contest-files/fanduel/current/projections.csv',
              # index=False
             )
    
    return None

In [10]:
def scrape_draftkings(**kwargs):

    path: str = '../data/current-draftkings.csv'
    if mode == 'single-game':
        path: str = path.replace('.csv', '-sg.csv')
    
    columns: dict[str, str] = {
        'Name': 'name',
        'Roster Position': 'pos',
        'TeamAbbrev': 'team',
        'Salary': 'salary'
    }
    
    inits_issues = {
        'SAS': 'SA',
        'PHX': 'PHO',
        'GSW': 'GS',
        'NOP': 'NO',
        'NYK': 'NY'
    }
    
    MIN_SAL: int = 1_000 if kwargs.get('drop_minimums', True) else 0
    
    keep_minimums: tuple[str,...] = tuple()
    drop_minimums: tuple[str,...] = tuple([
        name for name in (pd.read_csv(path, usecols=['Name','Salary']).pipe(lambda df_: df_.loc[df_['Salary'] == MIN_SAL]['Name'])) if name not in keep_minimums
    ])
    
    df: pd.DataFrame = (pd
                        .read_csv(path, usecols=columns)
                        .rename(columns,axis=1)
                        .pipe(lambda df_: df_.loc[(df_['pos']!='CPT') ])# For single game contests
                        .assign(
                            name=lambda df_: df_.name.str.replace('.','', regex=False),
                            pos=lambda df_: df_.pos
                            .str.replace('/[GF]/UTIL','', regex=True)
                            .str.replace('C/UTIL','C',regex=False)
                            .str.replace('/[GF]', '', regex=True)
                        )
                        .pipe(lambda df_: df_.loc[(df_['name'].isin(drop_minimums) == False)])
                        # .pipe(lambda df_: df_.loc[(df_['salary'] > 3_000)])
                       )
    
    name_issues: dict[str,str] = {
        'KJ Martin': 'Kenyon Martin',
        'KJ Martin Jr.': 'Kenyon Martin',
        'Guillermo Hernangomez': 'Willy Hernangomez',
    }
    
    
    fix_name: Callable[[str],str] = lambda name: ' '.join(name.split(' ')[:2])
    
    df['name'] = df['name'].map(lambda x: name_issues.get(x, fix_name(x)))
    df['team'] = df['team'].map(lambda x: inits_issues.get(x,x))
    
    df['input'] = tuple(zip(df['name'], df['team']))
    # df['input'] = df.loc[:,['name','team']].apply(tuple, axis=1) # Does the same thing
    df['output'] = df['input'].apply(lambda x: scrape_props(*x, 'draftkings'))
    
    df['fpts'] = df['output'].map(lambda x: x[0])
    df['e_fpts'] = df['output'].map(lambda x: x[1])
    
    
    for col in ('fpts', 'e_fpts'):
        df[f'{col}/$'] = 1000 * (df[col] / df['salary'])
    
    df['5x'] = 5 * (df['salary'] / 1000)
    df['value'] = df['fpts'] - df['5x']
    
    df = (df
          .loc[df['fpts']>0.0]
          .drop(['input', 'output', '5x'], axis=1)
          .assign(fpts_1k=lambda df_: 1000 * df_.fpts / df_.salary)
          .rename({'fpts_1k': 'fpts-1k'}, axis=1)
          .sort_values('value', ascending=False)
          .set_index('name')
          .round(2)
         )
    
    single_game: bool = 'sg' in path or len(df['team'].drop_duplicates()) == 2
    
    if single_game:
        df = (df
              .assign(
                  cpt_pts=lambda df_: df_.fpts * 1.5,
                  cpt_sal=lambda df_: df_.salary * 1.5,
                  cpt_fpts_1k=lambda df_: 1000 * df_.cpt_pts / df_.cpt_sal,
              )
              .assign(
                  cpt_sal=lambda df_: df_.cpt_sal.astype('int')
              )
              .round(2)
             )
    
    df.to_csv(f'../data/draftkings-props{"-sg" if single_game else ""}.csv')
    
    # # Save to optimizer
    df.to_csv('/home/deegs/devel/repos/nba-boxscores-git/nba-boxscores/data/2023-2024/contest-files/draftkings/current/projections.csv',
              # index=False
             )
    
    return None

In [11]:
# Temporary, just figuring out dynamics for now

def ScrapeProps(**kwargs):
    site: str = kwargs.get('site', 'draftkings')
    return scrape_fanduel(**kwargs) if site == 'fanduel' else scrape_draftkings(**kwargs)
    

In [12]:
def player_pool_distribution(df):
    df = (df
          .groupby('team')
          ['team']
          .agg(['count'])
          .set_axis(['num-players'], axis=1)
          .sort_values('num-players', ascending=False)
         )
    
    total_teams: int = len(pd
                           .read_csv(f'../data/current-draftkings{"-sg" if mode == "single-game" else ""}.csv', usecols=['TeamAbbrev'])
                           .rename({'TeamAbbrev': 'Team'}, axis=1)
                           ['Team']
                           .drop_duplicates()
                          )
    
    print(f'{len(df)} teams total...')
    print(f'Missing: {int(100*(1 - (len(df) / total_teams)))}% of teams...\n')
    
    return df

In [13]:
def output_box(msg: str, *args, **kwargs) ->  None:
    tb: str = ''.join(['   ', '-'*len(msg)])
    print(*[tb, f'   {msg}', tb], sep='\n')
    return


def load_slate(site: str, **kwargs):
    verbose: int = kwargs.get('verbose', 1)
    exclude = kwargs.get('exclude', list())
    drop = kwargs.get('drop', list())
    ret: pd.DataFrame = (pd
                         .read_csv(f'../data/{site}-props{"-sg" if mode == "single-game" else ""}.csv')
                         .pipe(lambda df_: df_.loc[df_['name'].isin(drop) == False])
                         .pipe(lambda df_: df_.loc[df_['team'].isin(exclude) == False])
                         .sort_values(by=kwargs.get('sort', 'fpts'), ascending=False)
                         .set_index('name')
                        )
    
    if verbose:
        msg = f'{len(ret)} total players'.upper()
        output_box(msg)
        print(player_pool_distribution(ret))
    
    return ret

def team_players(*args, **kwargs):
    df: pd.DataFrame = load_slate(site=site_)
    return {
        team: tuple(df
                    .loc[(df['team'] == team) & (df['value'] >= 0.0)]
                    .index
                   )
        for team in df['team'].drop_duplicates()
    }


def pos_value_players(site: str, *args, **kwargs) -> tuple[str,...]:
    ret: tuple[str,...] = tuple(load_slate(site=site, verbose=0, **kwargs)
                                .pipe(lambda df_: df_.loc[df_['value'] >= kwargs.get('value', 0.0)])
                                .sort_values('value', ascending=False)
                                .index
                               )
    
    msg = f'{len(ret)} total players'.upper()
    output_box(msg)
    return ret
# team_players()

In [14]:
import time
def output_times(func, **kwargs) -> None:
    """Wrapper function to print performance time in Xm Ys format"""
    start = time.perf_counter()
    func(**kwargs)
    stop = time.perf_counter()
    
    elapsed: float = (stop - start)/60.0
    
    elapsed_str: str = str(elapsed)
    minutes: int = int( elapsed_str.split('.')[0] )
    
    decimals: float = float( f'0.{elapsed_str.split(".")[1]}' )
    seconds: int = int(decimals * 60.0)
    
    performance_time: str = f'{minutes}m {seconds}s.'
    
    print(f"{func.__name__} performance time for {site.capitalize().replace('duel','Duel').replace('kings', 'Kings')}: {performance_time}\n")
    
    return None

In [15]:
# ScrapeProps(site=site)
last_update = pd.read_csv(f'../data/{site}-props{"-sg" if mode == "single-game" else ""}.csv').set_index('name')
output_times(ScrapeProps, site=site, drop_minimums=False, mute_touchdowns=False)

ScrapeProps performance time for FanDuel: 1m 41s.



In [16]:
df = load_slate(
    site,
    sort='value',
    # drop=[], # Late additions to injury report
    # exclude=['WAS', 'NY', 'CHI', 'TOR', 'OKC', 'UTA'] # Games that have already started
).drop('fpts-1k', axis=1)

   ----------------
   89 TOTAL PLAYERS
   ----------------
14 teams total...
Missing: -250% of teams...

      num-players
team             
CLE             8
IND             8
LAC             8
NO              8
OKC             8
ORL             8
ATL             7
PHO             6
TOR             6
DAL             5
HOU             5
MIL             5
CHA             4
MEM             3


In [17]:
updated_players = list(set(df.index).difference(set(last_update.index)))
output = ['The following players have been added:']
output += [f'    > {name_}' for name_ in updated_players]
if not updated_players:
    output = ['No players updated since last scrape.']
print(*output, sep='\n')

The following players have been added:
    > Trey Murphy
    > Chet Holmgren
    > Khris Middleton
    > Isaac Okoro
    > Shai Gilgeous-Alexander
    > LaMelo Ball
    > Grayson Allen
    > Cason Wallace
    > Jusuf Nurkic
    > Dennis Schroder
    > Saddiq Bey
    > Sam Merrill
    > Vince Williams
    > Jontay Porter
    > Josh Green
    > Mason Plumlee
    > Kevin Durant
    > Andrew Nembhard
    > Josh Giddey
    > Dillon Brooks
    > Jabari Smith
    > Jalen Williams
    > Scottie Barnes
    > Bogdan Bogdanovic
    > Grant Williams
    > Daniel Theis
    > Franz Wagner
    > Devin Booker
    > Kenrich Williams
    > Dejounte Murray
    > Caris LeVert
    > RJ Barrett
    > Buddy Hield
    > James Harden
    > Dean Wade
    > Luka Doncic
    > Russell Westbrook
    > Derrick Jones
    > Alperen Sengun
    > Jalen Johnson
    > Giannis Antetokounmpo
    > Damian Lillard
    > Brandon Ingram
    > Luguentz Dort
    > Bradley Beal
    > Brandon Miller
    > Donovan Mitchell
    > Col

In [18]:
# df.sort_values('fpts', ascending=False)
# df.sort_values('fpts/$', ascending=False)
df.sort_values('value', ascending=False)
# df.sort_values('salary')

,pos,salary,team,fpts,e_fpts,fpts/$,e_fpts/$,value
name,,,,,,,,
Luka Doncic,PG,12300,DAL,69.95,37.09,5.69,3.02,8.45
Alperen Sengun,C,9800,HOU,53.85,25.94,5.49,2.65,4.85
Jaren Jackson,PF/C,8600,MEM,47.35,24.02,5.51,2.79,4.35
Giannis Antetokounmpo,PF,12200,MIL,65.25,31.93,5.35,2.62,4.25
Donovan Mitchell,SG/PG,10100,CLE,54.35,28.45,5.38,2.82,3.85
James Harden,SG/PG,8500,LAC,46.35,23.52,5.45,2.77,3.85
Andrew Nembhard,SG/PG,5100,IND,27.25,12.93,5.34,2.54,1.75
Jarrett Allen,C/PF,8300,CLE,43.05,21.51,5.19,2.59,1.55
Damian Lillard,PG,8800,MIL,45.45,21.75,5.16,2.47,1.45


In [19]:
def flatten(nestedSeq) -> list[list[str,...], ...]:
    return [element for inner in nestedSeq for element in inner]

def get_top_names(df: pd.DataFrame, n=2, by='value') -> pd.DataFrame:
    """
    Returns only the top n players from each team by provided parameter, defaults to value
    """
    df = df.sort_values(by, ascending=False)
    top_names = {team: list(df.loc[df['team'] == team].index)[:n] for team in df['team'].drop_duplicates()}
    
    return df.loc[df.index.isin(flatten(list(top_names.values())))]

top_df = get_top_names(df, n=3)
# top_df

In [20]:
pos_dfs = dict()
for pos in ('PG', 'SG', 'SF', 'PF', 'C'):
    df[pos] = df['pos'].map(lambda pos_: int(pos in pos_))
    pos_dfs[pos] = df.loc[df[pos] == 1].drop(pos, axis=1).sort_values('value', ascending=False)
    df = df.drop(pos, axis=1)

In [21]:
pos_dfs['PG']

,pos,salary,team,fpts,e_fpts,fpts/$,e_fpts/$,value
name,,,,,,,,
Luka Doncic,PG,12300,DAL,69.95,37.09,5.69,3.02,8.45
Donovan Mitchell,SG/PG,10100,CLE,54.35,28.45,5.38,2.82,3.85
James Harden,SG/PG,8500,LAC,46.35,23.52,5.45,2.77,3.85
Andrew Nembhard,SG/PG,5100,IND,27.25,12.93,5.34,2.54,1.75
Damian Lillard,PG,8800,MIL,45.45,21.75,5.16,2.47,1.45
TJ McConnell,PG,5500,IND,28.25,13.40,5.14,2.44,0.75
LaMelo Ball,PG,9900,CHA,49.35,26.35,4.98,2.66,-0.15
Shai Gilgeous-Alexander,PG,11000,OKC,54.85,29.07,4.99,2.64,-0.15
Markelle Fultz,PG,5400,ORL,26.45,13.47,4.90,2.49,-0.55


In [22]:
pos_dfs['SG']

,pos,salary,team,fpts,e_fpts,fpts/$,e_fpts/$,value
name,,,,,,,,
Donovan Mitchell,SG/PG,10100,CLE,54.35,28.45,5.38,2.82,3.85
James Harden,SG/PG,8500,LAC,46.35,23.52,5.45,2.77,3.85
Andrew Nembhard,SG/PG,5100,IND,27.25,12.93,5.34,2.54,1.75
Luke Kennard,SG/SF,5700,MEM,28.45,13.48,4.99,2.36,-0.05
Dillon Brooks,SG/SF,5400,HOU,25.95,12.43,4.81,2.30,-1.05
Josh Green,SF/SG,4600,DAL,21.45,10.92,4.66,2.37,-1.55
Dejounte Murray,SG/PG,9200,ATL,44.35,21.01,4.82,2.28,-1.65
Devin Booker,SG,9700,PHO,46.65,25.13,4.81,2.59,-1.85
Bennedict Mathurin,SG/SF,5000,IND,22.95,11.16,4.59,2.23,-2.05


In [23]:
pos_dfs['SF']

,pos,salary,team,fpts,e_fpts,fpts/$,e_fpts/$,value
name,,,,,,,,
Scottie Barnes,PF/SF,9600,TOR,49.25,25.15,5.13,2.62,1.25
Kevin Durant,SF/PF,9800,PHO,50.05,25.55,5.11,2.61,1.05
RJ Barrett,SF,7400,TOR,37.55,19.71,5.07,2.66,0.55
Kawhi Leonard,SF/PF,8900,LAC,44.55,24.76,5.01,2.78,0.05
Luke Kennard,SG/SF,5700,MEM,28.45,13.48,4.99,2.36,-0.05
Brandon Ingram,SF/PF,8000,NO,39.65,21.01,4.96,2.63,-0.35
Jalen Johnson,SF/PF,7700,ATL,37.95,18.82,4.93,2.44,-0.55
Dillon Brooks,SG/SF,5400,HOU,25.95,12.43,4.81,2.30,-1.05
Derrick Jones,SF/PF,4700,DAL,22.15,11.37,4.71,2.42,-1.35


In [24]:
pos_dfs['PF']

,pos,salary,team,fpts,e_fpts,fpts/$,e_fpts/$,value
name,,,,,,,,
Jaren Jackson,PF/C,8600,MEM,47.35,24.02,5.51,2.79,4.35
Giannis Antetokounmpo,PF,12200,MIL,65.25,31.93,5.35,2.62,4.25
Jarrett Allen,C/PF,8300,CLE,43.05,21.51,5.19,2.59,1.55
Scottie Barnes,PF/SF,9600,TOR,49.25,25.15,5.13,2.62,1.25
Pascal Siakam,PF/C,7700,IND,39.55,20.16,5.14,2.62,1.05
Kevin Durant,SF/PF,9800,PHO,50.05,25.55,5.11,2.61,1.05
Kawhi Leonard,SF/PF,8900,LAC,44.55,24.76,5.01,2.78,0.05
Zion Williamson,PF,7900,NO,39.35,21.19,4.98,2.68,-0.15
Brandon Ingram,SF/PF,8000,NO,39.65,21.01,4.96,2.63,-0.35


In [25]:
pos_dfs['C']

,pos,salary,team,fpts,e_fpts,fpts/$,e_fpts/$,value
name,,,,,,,,
Alperen Sengun,C,9800,HOU,53.85,25.94,5.49,2.65,4.85
Jaren Jackson,PF/C,8600,MEM,47.35,24.02,5.51,2.79,4.35
Jarrett Allen,C/PF,8300,CLE,43.05,21.51,5.19,2.59,1.55
Pascal Siakam,PF/C,7700,IND,39.55,20.16,5.14,2.62,1.05
Nick Richards,C,6100,CHA,28.65,13.45,4.70,2.20,-1.85
Myles Turner,C,7200,IND,33.55,16.97,4.66,2.36,-2.45
Jonas Valanciunas,C,7100,NO,32.85,16.58,4.63,2.34,-2.65
Mason Plumlee,C,4900,LAC,21.75,10.94,4.44,2.23,-2.75
Clint Capela,C,6300,ATL,28.65,15.52,4.55,2.46,-2.85


In [26]:
# team_dfs = {team: df.loc[df['team'] == team] for team in df['team'].drop_duplicates()}
dict(sorted({team: df.loc[df['team'] == team].sort_values('value', ascending=False) for team in df['team'].drop_duplicates()}.items(), key=lambda item: item[0]))

{'ATL':                      pos  salary team   fpts  e_fpts  fpts/$  e_fpts/$  value
 name                                                                         
 Jalen Johnson      SF/PF    7700  ATL  37.95   18.82    4.93      2.44  -0.55
 Dejounte Murray    SG/PG    9200  ATL  44.35   21.01    4.82      2.28  -1.65
 Clint Capela           C    6300  ATL  28.65   15.52    4.55      2.46  -2.85
 Saddiq Bey         PF/SF    5700  ATL  24.35   12.80    4.27      2.25  -4.15
 Trae Young            PG   10000  ATL  44.75   21.55    4.47      2.16  -5.25
 Bogdan Bogdanovic  SF/SG    6400  ATL  24.25   12.40    3.79      1.94  -7.75
 Onyeka Okongwu      C/PF    6100  ATL  20.05   10.49    3.29      1.72 -10.45,
 'CHA':                pos  salary team   fpts  e_fpts  fpts/$  e_fpts/$  value
 name                                                                   
 LaMelo Ball     PG    9900  CHA  49.35   26.35    4.98      2.66  -0.15
 Nick Richards    C    6100  CHA  28.65   13.45    4.70